# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,duluth,46.7833,-92.1066,9.19,64,0,0.89,US,1707156203
1,1,chibougamau,49.9168,-74.3659,-9.21,92,0,2.06,CA,1707156204
2,2,iqaluit,63.7506,-68.5145,-20.15,77,100,1.03,CA,1707156204
3,3,stanley,54.8680,-1.6985,10.77,79,75,10.29,GB,1707156204
4,4,margaret river,-33.9500,115.0667,12.96,91,2,4.11,AU,1707156204


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height =600 ,
    size = "Humidity",
    alpha = 0.5,
    color = "City")


# Display the map
all_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
templow27= city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]
                            

# temphigh21 = templow27.loc[templow27["Max Temp"]>21]
# windspeed = temphigh21.loc[temphigh21["Wind Speed"]<4.5]
# zerocloud = windspeed.loc[windspeed["Cloudiness"]==0]


# Drop any rows with null values
ideal_weather = templow27.dropna()

# Display sample data
ideal_weather


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,bredasdorp,-34.5322,20.0403,23.20,68,0,4.03,ZA,1707156205
110,110,east london,-33.0153,27.9116,24.52,78,0,3.09,ZA,1707156222
112,112,hermanus,-34.4187,19.2345,22.11,82,0,4.36,ZA,1707156222
138,138,kailua-kona,19.6406,-155.9956,21.95,72,0,1.54,US,1707155970
203,203,ixtapa,20.7000,-105.2000,24.57,47,0,1.54,MX,1707156236
274,274,contai,21.7786,87.7536,21.78,76,0,3.40,IN,1707156345
284,284,peddapalli,18.6167,79.3667,25.47,48,0,2.91,IN,1707156346
288,288,syriam,16.7722,96.2378,23.06,83,0,2.57,MM,1707156347
395,395,nova sintra,14.8667,-24.7167,21.49,66,0,4.48,CV,1707156367
399,399,washim,20.1000,77.1500,24.77,33,0,1.66,IN,1707156368


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City","Country","Lat","Lng","Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,bredasdorp,ZA,-34.5322,20.0403,68,
110,east london,ZA,-33.0153,27.9116,78,
112,hermanus,ZA,-34.4187,19.2345,82,
138,kailua-kona,US,19.6406,-155.9956,72,
203,ixtapa,MX,20.7000,-105.2000,47,
274,contai,IN,21.7786,87.7536,76,
284,peddapalli,IN,18.6167,79.3667,48,
288,syriam,MM,16.7722,96.2378,83,
395,nova sintra,CV,14.8667,-24.7167,66,
399,washim,IN,20.1000,77.1500,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key,"categories":"accommodation.hotel"}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


bredasdorp - nearest hotel: Victoria Hotel
east london - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
kailua-kona - nearest hotel: Kona Seaside Hotel
ixtapa - nearest hotel: Ma. Cristina
contai - nearest hotel: No hotel found
peddapalli - nearest hotel: No hotel found
syriam - nearest hotel: Excel River View
nova sintra - nearest hotel: Residência Ka Dencho
washim - nearest hotel: No hotel found
garachico - nearest hotel: Residencia El Drago
tazacorte - nearest hotel: App Leyma
jeddah - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal


,City,Country,Lat,Lng,Humidity,Hotel Name
9,bredasdorp,ZA,-34.5322,20.0403,68,Victoria Hotel
110,east london,ZA,-33.0153,27.9116,78,No hotel found
112,hermanus,ZA,-34.4187,19.2345,82,Aloe guest house
138,kailua-kona,US,19.6406,-155.9956,72,Kona Seaside Hotel
203,ixtapa,MX,20.7000,-105.2000,47,Ma. Cristina
274,contai,IN,21.7786,87.7536,76,No hotel found
284,peddapalli,IN,18.6167,79.3667,48,No hotel found
288,syriam,MM,16.7722,96.2378,83,Excel River View
395,nova sintra,CV,14.8667,-24.7167,66,Residência Ka Dencho
399,washim,IN,20.1000,77.1500,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng","Lat",geo=True,tiles="OSM",frame_width=800,frame_height=600,
    size="Humidity",alpha=0.5,color="City",hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)